In [1]:
import torch

In [ ]:
!pip install simpletransformers

In [3]:
import numpy as np
import pandas as pd

In [5]:
from simpletransformers.language_representation import RepresentationModel

In [6]:
bert_large = RepresentationModel(
        model_type="bert",
        model_name="bert-large-uncased",
        use_cuda=True
    )

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForTextRepresentation: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTextRepresentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTextRepresentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:

bert_base = RepresentationModel(
        model_type="bert",
        model_name="bert-base-uncased",
        use_cuda=True
    )

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTextRepresentation: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTextRepresentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTextRepresentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
bert_tiny = RepresentationModel(
        model_type="bert",
        model_name="prajjwal1/bert-tiny",
        use_cuda=True
    )

Some weights of the model checkpoint at prajjwal1/bert-tiny were not used when initializing BertForTextRepresentation: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTextRepresentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTextRepresentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:

bert_mini = RepresentationModel(
        model_type="bert",
        model_name="prajjwal1/bert-mini",
        use_cuda=True
    )

Some weights of the model checkpoint at prajjwal1/bert-mini were not used when initializing BertForTextRepresentation: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTextRepresentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTextRepresentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
bert_small = RepresentationModel(
        model_type="bert",
        model_name="prajjwal1/bert-small",
        use_cuda=True
    )

Some weights of the model checkpoint at prajjwal1/bert-small were not used when initializing BertForTextRepresentation: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTextRepresentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTextRepresentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
# use one of the bert models to get word vectors

def get_word_vector(sent,word):
  sentences = []
  sentences.append(sent)
  word_vectors = bert_large.encode_sentences(sentences, combine_strategy=None)
  for i in range(len(sentences[0].split(' '))):
    if(sentences[0].split(' ')[i] == word):
      return word_vectors[0][i+1]

In [12]:
v = get_word_vector("The flower is beautiful","flower")

In [13]:
len(v)

1024

In [15]:
def get_effect_size(df1, df2, k="bias_prior_corrected"):
    diff = (df1[k].mean() - df2[k].mean())
    std_ = pd.concat([df1, df2], axis=0)[k].std() + 1e-8
    return diff / std_

In [16]:
def cosine_similarity(x, y):
  # if(x is not None):
  #   if(y is not None):
  #     num = np.dot(x, y)
  #     a = np.linalg.norm(x)
  #     b = np.linalg.norm(y)
  #     den = a*b
  #     res = num/den
  #     return res
  return np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))

In [17]:
def get_word_bias_scores(targets, A, B, sentences, group=True):
    wvs_targets = [
        (t, get_word_vector(sentence.replace("GGG", t).replace("XXX", "[MASK]"), t) )
        for sentence in sentences
        for t in targets
    ]
    wvs_A = [
        get_word_vector(sentence.replace("GGG", "[MASK]").replace("XXX", a), a)
        for sentence in sentences
        for a in A
    ]
    wvs_B = [
        get_word_vector(sentence.replace("GGG", "[MASK]").replace("XXX", b), b)
        for sentence in sentences
        for b in B
    ]
    df1 = pd.DataFrame([
        {"target": t, "score": cosine_similarity(wv, wva)}
        for wva in wvs_A
        for t, wv in wvs_targets
    ])
    if group: df1 = df1.groupby("target").mean()["score"].reset_index()
    df2 = pd.DataFrame([
        {"target": t, "score": cosine_similarity(wv, wvb)}
        for wvb in wvs_B
        for t, wv in wvs_targets
    ])
    if group: df2 = df2.groupby("target").mean()["score"].reset_index()
    df = df1.copy()
    df["bias_prior_corrected"] = df1["score"] - df2["score"]
    return df[["target", "bias_prior_corrected"]]

In [18]:
#target words
flower_words = ['aster', 'clover', 'hyacinth', 'marigold', 'poppy', 'azalea', 'crocus', 'iris', 'orchid', 'rose', 'bluebell', 'daffodil', 'lilac', 'pansy', 'tulip', 'buttercup', 'daisy', 'lily', 'peony', 'violet', 'carnation', 'gladiola',
'magnolia', 'petunia', 'zinnia']

insect_words = ['ant', 'caterpillar', 'flea', 'locust', 'spider', 'bedbug', 'centipede', 'fly', 'maggot', 'tarantula',
'bee', 'cockroach', 'gnat', 'mosquito', 'termite', 'beetle', 'cricket', 'hornet', 'moth', 'wasp', 'blackfly',
'dragonfly', 'horsefly', 'roach', 'weevil']


print(len(flower_words) == len(insect_words))

#attribute words
pleasant_words = ['caress', 'freedom', 'health', 'love', 'peace', 'cheer', 'friend', 'heaven', 'loyal', 'pleasure', 'diamond', 'gentle', 'honest', 'lucky', 'rainbow', 'diploma', 'gift', 'honor', 'miracle', 'sunrise', 'family',
'happy', 'laughter', 'paradise', 'vacation']


unpleasant_words = ['abuse', 'crash', 'filth', 'murder', 'sickness', 'accident', 'death', 'grief', 'poison', 'stink',
'assault', 'disaster', 'hatred', 'pollute', 'tragedy', 'divorce', 'jail', 'poverty', 'ugly', 'cancer', 'kill', 'rotten',
'vomit', 'agony', 'prison']

True


In [19]:
df1 = get_word_bias_scores(flower_words, pleasant_words,
                           unpleasant_words, ["GGG are XXX", "the GGG is XXX"], group=True)

df2 = get_word_bias_scores(insect_words, pleasant_words,
                           unpleasant_words, ["GGG are XXX", "the GGG is XXX"], group=True)

print("Flowers vs. Insects : ",get_effect_size(df1, df2))

Flowers vs. Insects :  0.516010487732804


In [20]:
#target words
instruments = ["bagpipe", "cello", "guitar", "lute", "trombone", "banjo", "clarinet", "harmonica",
               "mandolin", "trumpet", "bassoon", "drum", "harp", "oboe", "tuba", "bell", "fiddle", "harpsichord", "piano", "viola",
               "bongo", "flute", "horn", "saxophone", "violin"]

weapons = ["arrow", "club", "gun", "missile", "spear", "axe", "dagger", "harpoon", "pistol", "sword", "blade", "dynamite", "hatchet", "rifle", "tank",
           "bomb", "firearm", "knife", "shotgun", "teargas", "cannon", "grenade", "mace", "slingshot", "whip"]

print(len(instruments)==len(weapons))
#weapons[:len(instruments)]

#attribute words
pleasant_words = ['caress', 'freedom', 'health', 'love', 'peace', 'cheer', 'friend', 'heaven', 'loyal', 'pleasure', 'diamond', 'gentle', 'honest', 'lucky', 'rainbow', 'diploma', 'gift', 'honor', 'miracle', 'sunrise', 'family',
'happy', 'laughter', 'paradise', 'vacation']

unpleasant_words = ['abuse', 'crash', 'filth', 'murder', 'sickness', 'accident', 'death', 'grief', 'poison', 'stink',
'assault', 'disaster', 'hatred', 'pollute', 'tragedy', 'divorce', 'jail', 'poverty', 'ugly', 'cancer', 'kill', 'rotten',
'vomit', 'agony', 'prison']

True


In [21]:
df1 = get_word_bias_scores(instruments, pleasant_words,
                           unpleasant_words, ["GGG are XXX", "the GGG is XXX"], group=True)

df2 = get_word_bias_scores(weapons, pleasant_words,
                           unpleasant_words, ["GGG are XXX", "the GGG is XXX"], group=True)

print("Instruments vs. Weapons : ",get_effect_size(df1, df2))

Instruments vs. Weapons :  0.8425241112278351


In [22]:
#target words
european_american_nams = ["Adam", "Chip", "Harry", "Josh", "Roger", "Alan", "Frank", "Ian", "Justin", "Ryan", "Andrew", "Fred",
                           "Jack", "Matthew", "Stephen", "Brad", "Greg", "Jed", "Paul", "Todd", "Brandon", "Hank", "Jonathan", "Peter",
                           "Wilbur", "Amanda", "Courtney", "Heather", "Melanie", "Sara", "Amber", "Crystal", "Katie", "Meredith", "Shannon",
                          "Betsy", "Donna", "Kristin", "Nancy", "Stephanie", "Bobbie-Sue", "Ellen", "Lauren", "Peggy", "Sue-Ellen",
                           "Colleen", "Emily", "Megan", "Rachel", "Wendy"]

african_american_nams = ["Alonzo", "Jamel", "Lerone", "Percell", "Theo","Alphonse", "Jerome", "Leroy", "Rasaan", "Torrance", "Darnell", "Lamar",
                          "Lionel", "Rashaun", "Tyree", "Deion", "Lamont", "Malik", "Terrence", "Tyrone", "Everol", "Lavon", "Marcellus", "Terryl",
                          "Wardell", "Aiesha", "Lashelle", "Nichelle", "Shereen", "Temeka", "Ebony", "Latisha", "Shaniqua", "Tameisha", "Teretha",
                          'Jasmine', "Latonya", "Shanise", "Tanisha", "Tia", "Lakisha", "Latoya", "Sharise", "Tashika", "Yolanda", "Lashandra", "Malika",
                          "Shavonn", "Tawanda", "Yvette"]


#attribute words
european_american_names = [word.lower() for word in european_american_nams]

african_american_names = [word.lower() for word in african_american_nams]

print(len(european_american_names) == len(african_american_names))

pleasant_words = ['caress', 'freedom', 'health', 'love', 'peace', 'cheer', 'friend', 'heaven', 'loyal', 'pleasure', 'diamond', 'gentle', 'honest', 'lucky', 'rainbow', 'diploma', 'gift', 'honor', 'miracle', 'sunrise', 'family',
'happy', 'laughter', 'paradise', 'vacation']


unpleasant_words = ['abuse', 'crash', 'filth', 'murder', 'sickness', 'accident', 'death', 'grief', 'poison', 'stink',
'assault', 'disaster', 'hatred', 'pollute', 'tragedy', 'bomb', 'divorce', 'jail', 'poverty', 'ugly', 'cancer', 'evil', 'kill', 'rotten', 'vomit']

True


In [23]:
df1 = get_word_bias_scores(european_american_names, pleasant_words,
                           unpleasant_words, ["GGG are XXX", "the GGG is XXX"], group=True)

df2 = get_word_bias_scores(african_american_names, pleasant_words,
                           unpleasant_words, ["GGG are XXX", "the GGG is XXX"], group=True)

print("european_american_names vs. african_american_names : ",get_effect_size(df1, df2))

european_american_names vs. african_american_names :  0.335722894403222


In [24]:
#target words
european_american_nams = ["Brad", "Brendan", "Geoffrey", "Greg", "Brett", "Jay", "Matthew", "Neil",
 "Todd", "Allison", "Anne", "Carrie", "Emily", "Jill", "Laurie", "Kristen", "Meredith", "Sarah"]

african_american_nams = ["Darnell", "Hakim", "Jermaine", "Kareem", "Jamal", "Leroy", "Rasheed", "Tremayne", "Tyrone", "Aisha",
"Ebony", "Keisha", "Kenya", "Latonya", "Lakisha", "Latoya", "Tamika", "Tanisha"]

european_american_names = [word.lower() for word in european_american_nams]

african_american_names = [word.lower() for word in african_american_nams]

print(len(european_american_names) == len(african_american_names))
#attribute words

pleasant_words = ['caress', 'freedom', 'health', 'love', 'peace', 'cheer', 'friend', 'heaven', 'loyal', 'pleasure', 'diamond', 'gentle', 'honest', 'lucky', 'rainbow', 'diploma', 'gift', 'honor', 'miracle', 'sunrise', 'family',
'happy', 'laughter', 'paradise', 'vacation']


unpleasant_words = ['abuse', 'crash', 'filth', 'murder', 'sickness', 'accident', 'death', 'grief', 'poison', 'stink',
'assault', 'disaster', 'hatred', 'pollute', 'tragedy', 'bomb', 'divorce', 'jail', 'poverty', 'ugly', 'cancer', 'evil', 'kill', 'rotten', 'vomit']

True


In [25]:
df1 = get_word_bias_scores(european_american_names, pleasant_words,
                           unpleasant_words, ["GGG are XXX", "the GGG is XXX"], group=True)

df2 = get_word_bias_scores(african_american_names, pleasant_words,
                           unpleasant_words, ["GGG are XXX", "the GGG is XXX"], group=True)

print("european_american_names vs. african_american_names : ",get_effect_size(df1, df2))

european_american_names vs. african_american_names :  -0.1835364201484704


In [26]:
#target words
european_american_nams = ["Brad", "Brendan", "Geoffrey", "Greg", "Brett", "Jay", "Matthew", "Neil",
 "Todd", "Allison", "Anne", "Carrie", "Emily", "Jill", "Laurie", "Kristen", "Meredith", "Sarah"]

african_american_nams = ["Darnell", "Hakim", "Jermaine", "Kareem", "Jamal", "Leroy", "Rasheed", "Tremayne", "Tyrone", "Aisha",
"Ebony", "Keisha", "Kenya", "Latonya", "Lakisha", "Latoya", "Tamika", "Tanisha"]

european_american_names = [word.lower() for word in european_american_nams]

african_american_names = [word.lower() for word in african_american_nams]


print(len(european_american_names) == len(african_american_names))
#attribute words
pleasant = ["joy", "love", "peace", "wonderful", "pleasure", "friend", "laughter", "happy"]


unpleasant = ["agony", "terrible", "horrible", "nasty", "evil", "war", "awful", "failure"]

True


In [27]:
df1 = get_word_bias_scores(european_american_names, pleasant,
                           unpleasant, ["GGG are XXX", "the GGG is XXX"], group=True)

df2 = get_word_bias_scores(african_american_names, pleasant,
                           unpleasant, ["GGG are XXX", "the GGG is XXX"], group=True)

print("european_american_names vs. african_american_names : ",get_effect_size(df1, df2))

european_american_names vs. african_american_names :  0.45920339773295976


In [28]:
#target words
male_words = ['john', 'paul', 'mike', 'kevin', 'steve', 'greg', 'jeff', 'bill']

female_words = ['amy', 'joan', 'lisa', 'sarah', 'diana', 'kate', 'ann', 'donna']

print(len(male_words) == len(female_words))

#attribute words
career_words = ['executive', 'management', 'professional', 'corporation', 'salary', 'office', 'business', 'career']

family_words = ['home', 'parents', 'children', 'family', 'cousins', 'marriage', 'wedding', 'relatives']

True


In [29]:
df1 = get_word_bias_scores(male_words, career_words,
                           family_words, ["GGG likes XXX", "GGG like XXX", "GGG is interested in XXX"], group=True)

df2 = get_word_bias_scores(female_words, family_words,
                           family_words, ["GGG likes XXX", "GGG like XXX", "GGG is interested in XXX"], group=True)

print("Career vs Family : ",get_effect_size(df1, df2))

Career vs Family :  1.0305368842590559


In [30]:
#attribute words
math_words = ["math", "algebra", "geometry", "calculus", "equations", "computation", "numbers", "addition"]

art_words = ["poetry", "art", "dance", "shakespear", "literature", "novels", "symphony", "drama"]

print(len(math_words) == len(art_words))

#target words
male_words = ["male", "man", "boy", "brother", "son", "he", "his", "him"]

female_words = ["female", "woman", "girl", "sister", "daughter", "she", "her", "hers"]

True


In [31]:
df1 = get_word_bias_scores(male_words, math_words,
                           art_words, ["GGG likes XXX", "GGG like XXX", "GGG is interested in XXX"], group=True)

df2 = get_word_bias_scores(female_words, math_words,
                           art_words, ["GGG likes XXX", "GGG like XXX", "GGG is interested in XXX"], group=True)

print("Math vs Art : ",get_effect_size(df1, df2))

Math vs Art :  -0.133460777882017


In [32]:
#target words
male_words = ["brother", "father", "uncle", "grandfather", "son", "he", "his", "him"]

female_words = ["sister", "mother", "aunt", "grandmother", "daughter", "she", "hers", "her"]

print(len(male_words) == len(female_words))
#attribute words

science_words = ["science", "technology", "physics", "chemistry", "einstein", "nasa", "experiments", "astronomy"]

art_words = ["poetry", "art", "shakespeare", "dance", "literature", "novel", "symphony", "drama"]

True


In [33]:
df1 = get_word_bias_scores(male_words, science_words,
                           art_words, ["GGG likes XXX", "GGG like XXX", "GGG is interested in XXX"], group=True)

df2 = get_word_bias_scores(female_words, science_words,
                           art_words, ["GGG likes XXX", "GGG like XXX", "GGG is interested in XXX"], group=True)

print("Science vs Art : ",get_effect_size(df1, df2))

Science vs Art :  -0.09244466250875084


In [34]:
#target words
mental_disease = ["sad", "hopeless", "gloomy", "tearful", "miserable", "depressed"]

physical_disease = ["sick", "illness", "influenza", "disease", "virus", "cancer"]
print(len(mental_disease) == len(physical_disease))
#attribute words
temporary = ["impermanent", "unstable", "variable", "fleeting", "short-term", "brief", "occasional"]

permanent = ["stable", "always", "constant", "persistent", "chronic", "prolonged", "forever"]

True


In [35]:
df1 = get_word_bias_scores(mental_disease, temporary,
                           permanent, ["GGG are XXX", "the GGG is XXX"], group=True)

df2 = get_word_bias_scores(physical_disease, temporary,
                           permanent, ["GGG are XXX", "the GGG is XXX"], group=True)

print("Mental v/s Physical disease : ",get_effect_size(df1, df2))

Mental v/s Physical disease :  1.039846129247743


In [36]:
#target words
young_people_names = ["tiffany", "michelle", "cindy", "kristy", "brad", "eric", "joey", "billy"]

old_people_names = ["ethel", "bernice", "gertrude", "agnes", "cecil", "wilbert", "mortimer", "edgar"]

print(len(young_people_names) == len(old_people_names))
#attribute words
pleasant = ["joy", "love", "peace", "wonderful", "pleasure", "friend", "laughter", "happy"]


unpleasant = ["agony", "terrible", "horrible", "nasty", "evil", "war", "awful", "failure"]

True


In [37]:
df1 = get_word_bias_scores(young_people_names, pleasant,
                           unpleasant, ["GGG are XXX", "the GGG is XXX"], group=True)

df2 = get_word_bias_scores(old_people_names, pleasant,
                           unpleasant, ["GGG are XXX", "the GGG is XXX"], group=True)

print("young_people_names vs. old_people_names : ",get_effect_size(df1, df2))

young_people_names vs. old_people_names :  -0.25021059949101954
